In [ ]:
BigData. Фреймворк Apache Spark

In [ ]:
Урок 4. Машинное обучение на pySpark на примере линейной регрессии

In [ ]:
Работа на уроке: https://colab.research.google.com/drive/18kCmFr3QdlooCC0TUm9teaukbIX26MLY?usp=sharing
Joblib: https://colab.research.google.com/drive/1esnt7oWobjet5Yj1rbFKBLUyA1uG5QTa?usp=sharing
Estimators: https://colab.research.google.com/drive/14XmbKrZOx5IOI5OXY26VeGjzAkMJAiut?usp=sharing
Transformers: https://colab.research.google.com/drive/1dKsDj1Kc7acFz8YC9_I-4utYJdU_4D9P?usp=sharing
HW - перепишите всё на spark (без pandas и sklaarn), код моментами немного не очевидный (чтобы сложнее было переписать 1к1), 
но на самом деле простой: https://colab.research.google.com/drive/1_JScmu1V43DJ44Q2zwabWd2FDp0TVFTF#scrollTo=zDyEK4QgMH3f

In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "C2hfVwCGAwt9",
    "outputId": "7c969092-fcac-4643-b08d-d58a32a571d7"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\n",
      "Collecting pyspark\n",
      "  Downloading pyspark-3.3.1.tar.gz (281.4 MB)\n",
      "\u001b[2K     \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m281.4/281.4 MB\u001b[0m \u001b[31m4.7 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
      "\u001b[?25h  Preparing metadata (setup.py) ... \u001b[?25l\u001b[?25hdone\n",
      "Collecting py4j==0.10.9.5\n",
      "  Downloading py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)\n",
      "\u001b[2K     \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m199.7/199.7 KB\u001b[0m \u001b[31m24.2 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
      "\u001b[?25hBuilding wheels for collected packages: pyspark\n",
      "  Building wheel for pyspark (setup.py) ... \u001b[?25l\u001b[?25hdone\n",
      "  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=0e74b9dc220ed2b5cb29a64b980d99be1a9117d1ea56bafab524535bb5c26958\n",
      "  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598\n",
      "Successfully built pyspark\n",
      "Installing collected packages: py4j, pyspark\n",
      "Successfully installed py4j-0.10.9.5 pyspark-3.3.1\n"
     ]
    }
   ],
   "source": [
    "pip install pyspark"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "E3Q9g_UyNxS6"
   },
   "outputs": [],
   "source": [
    "from pyspark.sql import SparkSession\n",
    "\n",
    "\n",
    "spark = SparkSession.builder\\\n",
    "    .master(\"local[2]\")\\\n",
    "    .appName(\"Lesson_5\")\\\n",
    "    .config(\"spark.executor.instances\",2)\\\n",
    "    .config(\"spark.executor.memory\",'2g')\\\n",
    "    .config(\"spark.executor.cores\",1)\\\n",
    "    .getOrCreate()\n",
    "    \n",
    "sc = spark.sparkContext"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "dnXLtjUr_FkX"
   },
   "source": [
    "# Оконные функции"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "JGioDDCQ_V_-"
   },
   "source": [
    "Оконная функция - функция, которая работает с выделенным набором строк (окном, партицией) и выполняет вычисление для этого набора строк в отдельном столбце. \n",
    "\n",
    "Партиции (окна из набора строк) - это набор строк, указанный для оконной функции по одному из столбцов или группе столбцов таблицы. Партиции для каждой оконной функции в запросе могут быть разделены по различным колонкам таблицы."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "b6tdUDN5_Ee0",
    "outputId": "656fe78c-bc07-4aef-cec3-de9a6b6e4625"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- Product: string (nullable = true)\n",
      " |-- Amount: long (nullable = true)\n",
      " |-- Country: string (nullable = true)\n",
      "\n",
      "+-------+------+-------+\n",
      "|Product|Amount|Country|\n",
      "+-------+------+-------+\n",
      "|Banana |1000  |USA    |\n",
      "|Carrots|1500  |USA    |\n",
      "|Beans  |1600  |USA    |\n",
      "|Orange |2000  |USA    |\n",
      "|Orange |2000  |USA    |\n",
      "|Banana |400   |China  |\n",
      "|Carrots|1200  |China  |\n",
      "|Beans  |1500  |China  |\n",
      "|Orange |4000  |China  |\n",
      "|Banana |2000  |Canada |\n",
      "|Carrots|2000  |Canada |\n",
      "|Beans  |2000  |Mexico |\n",
      "+-------+------+-------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# Предположим, что есть таблицы:\n",
    "from pyspark.sql import functions as F\n",
    "from pyspark.sql.types import *\n",
    "\n",
    "data =[\n",
    "    (\"Banana\",1000,\"USA\"), (\"Carrots\",1500,\"USA\"), (\"Beans\",1600,\"USA\"),\\\n",
    "    (\"Orange\",2000,\"USA\"), (\"Orange\",2000,\"USA\"), (\"Banana\",400,\"China\"),\\\n",
    "    (\"Carrots\",1200,\"China\"), (\"Beans\",1500,\"China\"), (\"Orange\",4000,\"China\"),\\\n",
    "    (\"Banana\",2000,\"Canada\"), (\"Carrots\",2000,\"Canada\"), (\"Beans\",2000,\"Mexico\")\n",
    "    ]\n",
    "columns = [\"Product\",\"Amount\", \"Country\"]\n",
    "\n",
    "df = spark.createDataFrame(data=data, schema=columns)\n",
    "df.printSchema()\n",
    "df.show(truncate=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "jhq8wrA8BNia",
    "outputId": "371a5ec3-32c2-46a6-ffef-e8f616f28d13"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.\n",
      "  warnings.warn(\"Deprecated in 2.0, use createOrReplaceTempView instead.\", FutureWarning)\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-------+------+-------+---+\n",
      "|Product|Amount|Country| rn|\n",
      "+-------+------+-------+---+\n",
      "| Banana|  2000| Canada|  1|\n",
      "|Carrots|  2000| Canada|  2|\n",
      "| Banana|   400|  China|  1|\n",
      "|Carrots|  1200|  China|  2|\n",
      "|  Beans|  1500|  China|  3|\n",
      "| Orange|  4000|  China|  4|\n",
      "|  Beans|  2000| Mexico|  1|\n",
      "| Banana|  1000|    USA|  1|\n",
      "|Carrots|  1500|    USA|  2|\n",
      "|  Beans|  1600|    USA|  3|\n",
      "| Orange|  2000|    USA|  4|\n",
      "| Orange|  2000|    USA|  5|\n",
      "+-------+------+-------+---+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# в sql оконная функция записывается следующим образом\n",
    "df.registerTempTable('df')\n",
    "\n",
    "spark.sql('''\n",
    "select *, \n",
    "row_number() over( partition by Country order by Amount ) as rn from df\n",
    "''').show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "9grWEK6CBbBK",
    "outputId": "0afaf53c-8766-40a0-d9b2-1a16b62a2b4c"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-------+------+-------+---+\n",
      "|Product|Amount|Country| rn|\n",
      "+-------+------+-------+---+\n",
      "| Banana|  2000| Canada|  1|\n",
      "|Carrots|  2000| Canada|  2|\n",
      "| Banana|   400|  China|  1|\n",
      "|Carrots|  1200|  China|  2|\n",
      "|  Beans|  1500|  China|  3|\n",
      "| Orange|  4000|  China|  4|\n",
      "|  Beans|  2000| Mexico|  1|\n",
      "| Banana|  1000|    USA|  1|\n",
      "|Carrots|  1500|    USA|  2|\n",
      "|  Beans|  1600|    USA|  3|\n",
      "| Orange|  2000|    USA|  4|\n",
      "| Orange|  2000|    USA|  5|\n",
      "+-------+------+-------+---+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# в спарке следующим образом\n",
    "from pyspark.sql import Window\n",
    "\n",
    "\n",
    "windSpec = Window()\\\n",
    "    .partitionBy('Country')\\\n",
    "    .orderBy('Amount')\n",
    "    # .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)\n",
    "\n",
    "df.withColumn('rn', F.row_number().over(windSpec)).show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "dVGNGR7pN1KC"
   },
   "source": [
    "# Самостоятельная работа к уроку\n",
    "На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "agigNChqOHnK"
   },
   "source": [
    "## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)\n",
    "\n",
    " Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):\n",
    "*   date of sale\n",
    "*   price\n",
    "*   property type\n",
    "*   number of bedrooms\n",
    "*   4digit postcode"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "i-36e65zdEEh",
    "outputId": "248f9bd4-6556-41f3-8ad1-b0fb03a6ebd9"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "--2023-01-23 12:12:59--  https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07\n",
      "Resolving drive.google.com (drive.google.com)... 173.194.217.100, 173.194.217.139, 173.194.217.113, ...\n",
      "Connecting to drive.google.com (drive.google.com)|173.194.217.100|:443... connected.\n",
      "HTTP request sent, awaiting response... 303 See Other\n",
      "Location: https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/98tto341fq2o32hfickf06864c3c2rmh/1674475950000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=fc78e771-fe5b-4a7f-b34d-0919fb974561 [following]\n",
      "Warning: wildcards not supported in HTTP.\n",
      "--2023-01-23 12:12:59--  https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/98tto341fq2o32hfickf06864c3c2rmh/1674475950000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=fc78e771-fe5b-4a7f-b34d-0919fb974561\n",
      "Resolving doc-0c-5k-docs.googleusercontent.com (doc-0c-5k-docs.googleusercontent.com)... 142.251.162.132, 2607:f8b0:400c:c0b::84\n",
      "Connecting to doc-0c-5k-docs.googleusercontent.com (doc-0c-5k-docs.googleusercontent.com)|142.251.162.132|:443... connected.\n",
      "HTTP request sent, awaiting response... 200 OK\n",
      "Length: 1505497 (1.4M) [text/csv]\n",
      "Saving to: ‘raw_sales.csv’\n",
      "\n",
      "raw_sales.csv       100%[===================>]   1.44M  --.-KB/s    in 0.01s   \n",
      "\n",
      "2023-01-23 12:12:59 (129 MB/s) - ‘raw_sales.csv’ saved [1505497/1505497]\n",
      "\n"
     ]
    }
   ],
   "source": [
    "!wget 'https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07' -O raw_sales.csv"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "WpWic7isdH6R",
    "outputId": "85e48fc7-9156-4950-bb37-713382be1929"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "root\n",
      " |-- datesold: timestamp (nullable = true)\n",
      " |-- postcode: integer (nullable = true)\n",
      " |-- price: integer (nullable = true)\n",
      " |-- propertyType: string (nullable = true)\n",
      " |-- bedrooms: integer (nullable = true)\n",
      "\n",
      "+-------------------+--------+-------+------------+--------+\n",
      "|datesold           |postcode|price  |propertyType|bedrooms|\n",
      "+-------------------+--------+-------+------------+--------+\n",
      "|2007-02-07 00:00:00|2607    |525000 |house       |4       |\n",
      "|2007-02-27 00:00:00|2906    |290000 |house       |3       |\n",
      "|2007-03-07 00:00:00|2905    |328000 |house       |3       |\n",
      "|2007-03-09 00:00:00|2905    |380000 |house       |4       |\n",
      "|2007-03-21 00:00:00|2906    |310000 |house       |3       |\n",
      "|2007-04-04 00:00:00|2905    |465000 |house       |4       |\n",
      "|2007-04-24 00:00:00|2607    |399000 |house       |3       |\n",
      "|2007-04-30 00:00:00|2606    |1530000|house       |4       |\n",
      "|2007-05-24 00:00:00|2902    |359000 |house       |3       |\n",
      "|2007-05-25 00:00:00|2906    |320000 |house       |3       |\n",
      "|2007-06-26 00:00:00|2902    |385000 |house       |3       |\n",
      "|2007-06-27 00:00:00|2906    |305000 |house       |3       |\n",
      "|2007-06-27 00:00:00|2612    |850000 |house       |4       |\n",
      "|2007-06-28 00:00:00|2904    |765000 |house       |4       |\n",
      "|2007-06-30 00:00:00|2615    |517000 |house       |4       |\n",
      "|2007-07-02 00:00:00|2914    |800000 |house       |5       |\n",
      "|2007-07-03 00:00:00|2906    |336000 |house       |3       |\n",
      "|2007-07-06 00:00:00|2615    |535000 |house       |5       |\n",
      "|2007-07-07 00:00:00|2602    |900000 |house       |4       |\n",
      "|2007-07-08 00:00:00|2600    |327000 |house       |1       |\n",
      "+-------------------+--------+-------+------------+--------+\n",
      "only showing top 20 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)\n",
    "df.printSchema()\n",
    "df.show(truncate=False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "xisyFowtQgx-"
   },
   "source": [
    "## Задание 1\n",
    "Добавьте к таблице следующие поля:\n",
    "*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)\n",
    "*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)\n",
    "*  Стоимость последнего проданного дома до текущего\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "7fGJJreDczDK"
   },
   "outputs": [],
   "source": [
    "from pyspark.sql import Window\n",
    "import pyspark.sql.functions as F\n",
    "from pyspark.sql.functions import col, asc,desc\n",
    "\n",
    "windSpec_before = Window()\\\n",
    "    .partitionBy('postcode')\\\n",
    "    .orderBy('datesold')\n",
    "\n",
    "df = df.withColumn('average_price_before', F.avg('price').over(windSpec_before))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "QOT40OOb5TZ6"
   },
   "outputs": [],
   "source": [
    "windSpec_after = Window()\\\n",
    "    .partitionBy('postcode')\\\n",
    "    .orderBy(col('datesold').desc())\n",
    "\n",
    "df = df.withColumn('average_price_after', F.avg('price').over(windSpec_after))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "oK9MgruF_jsO"
   },
   "outputs": [],
   "source": [
    "df = df.withColumn('previous_sold_price', F.lag('price', 1).over(windSpec_before))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "-goBnh4NF-cJ",
    "outputId": "7e20c2f4-b296-45d0-b926-8e37872b82cf"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-------------------+--------+-------+------------+--------+--------------------+-------------------+-------------------+\n",
      "|           datesold|postcode|  price|propertyType|bedrooms|average_price_before|average_price_after|previous_sold_price|\n",
      "+-------------------+--------+-------+------------+--------+--------------------+-------------------+-------------------+\n",
      "|2007-07-08 00:00:00|    2600| 327000|       house|       1|            327000.0| 1028204.3785488959|               null|\n",
      "|2007-08-16 00:00:00|    2600| 790000|       house|       4|            558500.0| 1029312.1263823065|             327000|\n",
      "|2007-12-05 00:00:00|    2600| 825000|       house|       3|   647333.3333333334| 1029690.7848101265|             790000|\n",
      "|2008-01-21 00:00:00|    2600| 315000|        unit|       1|            564250.0|  1030015.175911252|             825000|\n",
      "|2008-04-24 00:00:00|    2600| 292500|       house|       1|            509900.0| 1031150.1206349207|             315000|\n",
      "|2008-05-30 00:00:00|    2600| 329000|        unit|       2|            479750.0| 1032324.4451510333|             292500|\n",
      "|2008-06-19 00:00:00|    2600| 765000|       house|       5|            520500.0| 1033444.3885350318|             329000|\n",
      "|2008-07-29 00:00:00|    2600| 927000|       house|       4|            571312.5| 1033872.5295055822|             765000|\n",
      "|2008-09-02 00:00:00|    2600|1380000|       house|       5|   661166.6666666666| 1034043.2523961661|             927000|\n",
      "|2008-09-08 00:00:00|    2600| 740000|       house|       3|            669050.0|       1033489.7216|            1380000|\n",
      "+-------------------+--------+-------+------------+--------+--------------------+-------------------+-------------------+\n",
      "only showing top 10 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df.show(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "Qvh2x6_8YU3F"
   },
   "source": [
    "## Задание 2\n",
    "В итоге у вас таблица с колонками (или нечто похожее):\n",
    "*   price\n",
    "*   Среднегодовая цена\n",
    "*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)\n",
    "*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)\n",
    "*  Стоимость последнего проданного дома до текущего ((1 балл)\n",
    "*  и др.\n",
    "\n",
    "Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "2eV6P_dNgXhI",
    "outputId": "6caf7901-9435-4043-9d48-e793ffd6d6df"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-------------------------------+\n",
      "|approx_count_distinct(postcode)|\n",
      "+-------------------------------+\n",
      "|                             27|\n",
      "+-------------------------------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df.select(F.approx_count_distinct('postcode')).show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "LEl8YHuvEVil",
    "outputId": "426bdff8-7fe1-4657-fc2a-8e006d64036d"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----------------------------------+\n",
      "|approx_count_distinct(propertyType)|\n",
      "+-----------------------------------+\n",
      "|                                  2|\n",
      "+-----------------------------------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df.select(F.approx_count_distinct('propertyType')).show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "NbvISPGoEYyu",
    "outputId": "a53552d4-2703-4b15-e02f-1e419eb8a59f"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-------------------------------+\n",
      "|approx_count_distinct(bedrooms)|\n",
      "+-------------------------------+\n",
      "|                              6|\n",
      "+-------------------------------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df.select(F.approx_count_distinct('bedrooms')).show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "pmSZTI9PAwQb"
   },
   "source": [
    "# Задание 3\n",
    "SQL like case when или if elif else\n",
    "\n",
    "Создайте колонку, в которой в которой будет отображаться \"+\", \"-\" или \"=\", если \"Средняя стомость 10 проданных домов до текущего в том же районе\" больше, меньше или равно \"Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)\", соотвественно.\n",
    "\n",
    "Если одно из полей Null, запишите в эту колонку \"Нет данных\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "-rfOBthog-2Q",
    "outputId": "4e211620-a586-4a02-e20b-be00afbc2209"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-------------------+--------+-------+------------+--------+--------------------+-------------------+-------------------+------+\n",
      "|           datesold|postcode|  price|propertyType|bedrooms|average_price_before|average_price_after|previous_sold_price|ifelse|\n",
      "+-------------------+--------+-------+------------+--------+--------------------+-------------------+-------------------+------+\n",
      "|2007-07-08 00:00:00|    2600| 327000|       house|       1|            327000.0| 1028204.3785488959|               null|     -|\n",
      "|2007-08-16 00:00:00|    2600| 790000|       house|       4|            558500.0| 1029312.1263823065|             327000|     -|\n",
      "|2007-12-05 00:00:00|    2600| 825000|       house|       3|   647333.3333333334| 1029690.7848101265|             790000|     -|\n",
      "|2008-01-21 00:00:00|    2600| 315000|        unit|       1|            564250.0|  1030015.175911252|             825000|     -|\n",
      "|2008-04-24 00:00:00|    2600| 292500|       house|       1|            509900.0| 1031150.1206349207|             315000|     -|\n",
      "|2008-05-30 00:00:00|    2600| 329000|        unit|       2|            479750.0| 1032324.4451510333|             292500|     -|\n",
      "|2008-06-19 00:00:00|    2600| 765000|       house|       5|            520500.0| 1033444.3885350318|             329000|     -|\n",
      "|2008-07-29 00:00:00|    2600| 927000|       house|       4|            571312.5| 1033872.5295055822|             765000|     -|\n",
      "|2008-09-02 00:00:00|    2600|1380000|       house|       5|   661166.6666666666| 1034043.2523961661|             927000|     -|\n",
      "|2008-09-08 00:00:00|    2600| 740000|       house|       3|            669050.0|       1033489.7216|            1380000|     -|\n",
      "+-------------------+--------+-------+------------+--------+--------------------+-------------------+-------------------+------+\n",
      "only showing top 10 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df.withColumn('ifelse', F.when(F.col('postcode').isNull() | F.col('price').isNull(), F.lit('Нет данных')).when(F.col('average_price_before') > F.col('average_price_after'), F.lit('+')).when(F.col('average_price_before') < F.col('average_price_after'), '-').otherwise('=')).show(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "k2q7HSEqWBVn"
   },
   "source": [
    "# Задание 4"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "c3pfUThFQtE6",
    "outputId": "42111b69-1497-4200-c91d-f571f9a31143"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "df1\n",
      "+-----+-------+-------+\n",
      "|key_1|value_1|value_2|\n",
      "+-----+-------+-------+\n",
      "|  abc|     10|     20|\n",
      "|  def|    100|    300|\n",
      "+-----+-------+-------+\n",
      "\n",
      "df2\n",
      "+-----+-------+-------+\n",
      "|key_2|value_1|value_2|\n",
      "+-----+-------+-------+\n",
      "|  abc|    5.5|    2.2|\n",
      "|  xyz|   10.1|   13.5|\n",
      "+-----+-------+-------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# Создаём датасет для примеров\n",
    "dataset_1 = [\n",
    "  {\n",
    "    'key_1' : 'abc',\n",
    "    'value_1' : 10,\n",
    "    'value_2' : 20\n",
    "  },\n",
    "  {\n",
    "    'key_1' : 'def',\n",
    "    'value_1' : 100,\n",
    "    'value_2' : 300\n",
    "  }\n",
    "]\n",
    "\n",
    "dataset_2 = [\n",
    "  {\n",
    "    'key_2' : 'abc',\n",
    "    'value_1' : 5.5,\n",
    "    'value_2' : 2.2\n",
    "  },\n",
    "  {\n",
    "    'key_2' : 'xyz',\n",
    "    'value_1' : 10.1,\n",
    "    'value_2' : 13.5\n",
    "  }\n",
    "]\n",
    "\n",
    "df1 = spark.createDataFrame(dataset_1)\n",
    "print('df1')\n",
    "df1.show()\n",
    "\n",
    "df2 = spark.createDataFrame(dataset_2)\n",
    "print('df2')\n",
    "df2.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "o0E_1iKeWFeY"
   },
   "source": [
    "## Задание 4.1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "od3V0xa4V7U8"
   },
   "outputs": [],
   "source": [
    "# Создайте джойн, чтобы получить следующую таблицу\n",
    "# +---+-------+-------+\n",
    "# |key|value_1|value_2|\n",
    "# +---+-------+-------+\n",
    "# |abc|     10|     20|\n",
    "# +---+-------+-------+"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "BRDcZgNKjZru",
    "outputId": "460df843-9465-48a0-81a4-129e7527c508"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----+-------+-------+\n",
      "|key_1|value_1|value_2|\n",
      "+-----+-------+-------+\n",
      "|  abc|     10|     20|\n",
      "+-----+-------+-------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df1.join(df2, on=df1.key_1 == df2.key_2, how='semi').show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "qrwYa2ZAWYmF"
   },
   "source": [
    "## Задание 4.2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "xEb-gGG8WQST"
   },
   "outputs": [],
   "source": [
    "# Создайте джойн, чтобы получить следующую таблицу\n",
    "# +---+-------+-------+\n",
    "# |key|value_1|value_2|\n",
    "# +---+-------+-------+\n",
    "# |def|    100|    300|\n",
    "# +---+-------+-------+"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "IKM1D_FIjBc9",
    "outputId": "05ee1dfe-4182-4017-d28e-9a9b14ed0352"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----+-------+-------+\n",
      "|key_1|value_1|value_2|\n",
      "+-----+-------+-------+\n",
      "|  def|    100|    300|\n",
      "+-----+-------+-------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df1.join(df2, on=df1.key_1 == df2.key_2, how='anti').show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "GbzsGu30WZTb"
   },
   "source": [
    "## Задание 4.3"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "XJ84QTURWWY_"
   },
   "outputs": [],
   "source": [
    "# Создайте Inner джойн с условиями ---hidden---, для df1 и df2, соответсвенно\n",
    "# В  итоге получится таблица\n",
    "# +---+-------+-------+---+-------+-------+\n",
    "# |key|value_1|value_2|key|value_1|value_2|\n",
    "# +---+-------+-------+---+-------+-------+\n",
    "# |abc|     10|     20|abc|    5.5|    2.2|\n",
    "# |abc|     10|     20|xyz|   10.1|   13.5|\n",
    "# |def|    100|    300|abc|    5.5|    2.2|\n",
    "# |def|    100|    300|xyz|   10.1|   13.5|\n",
    "# +---+-------+-------+---+-------+-------+"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "2zzGUJN7inrF",
    "outputId": "f86f4459-06a1-4ef4-bfd4-9d5cab618763"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "+-----+-------+-------+-----+-------+-------+\n",
      "|key_1|value_1|value_2|key_2|value_1|value_2|\n",
      "+-----+-------+-------+-----+-------+-------+\n",
      "|  abc|     10|     20|  abc|    5.5|    2.2|\n",
      "|  abc|     10|     20|  xyz|   10.1|   13.5|\n",
      "|  def|    100|    300|  abc|    5.5|    2.2|\n",
      "|  def|    100|    300|  xyz|   10.1|   13.5|\n",
      "+-----+-------+-------+-----+-------+-------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "df1.join(df2, on=df1.value_2 > df2.value_2, how='inner').show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "PNwKGLKri4xr"
   },
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.15"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}

In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "source": [
        "!pip install pyspark"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "AszXfjLrTDqJ",
        "outputId": "c72f1f00-663d-41fe-984f-dc904ab9c2a0"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\n",
            "Requirement already satisfied: pyspark in /usr/local/lib/python3.10/dist-packages (3.4.0)\n",
            "Requirement already satisfied: py4j==0.10.9.7 in /usr/local/lib/python3.10/dist-packages (from pyspark) (0.10.9.7)\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "E3Q9g_UyNxS6"
      },
      "source": [
        "from pyspark.sql import SparkSession\n",
        "spark = SparkSession.builder\\\n",
        "    .master(\"local[2]\")\\\n",
        "    .appName(\"Lesson_4\")\\\n",
        "    .config(\"spark.executor.instances\",2)\\\n",
        "    .config(\"spark.executor.memory\",'2g')\\\n",
        "    .config(\"spark.executor.cores\",1)\\\n",
        "    .getOrCreate()\n",
        "sc = spark.sparkContext"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "dVGNGR7pN1KC"
      },
      "source": [
        "# Самостоятельная работа к уроку 4\n",
        "На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "agigNChqOHnK"
      },
      "source": [
        "## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)\n",
        "\n",
        " Каждая строчка это продажа жилья, которая состоит из следующих полей:\n",
        "*   ```date of sale``` - дата продажи\n",
        "*   ```price``` - цена\n",
        "*   ```property type``` - тип недвижимости\n",
        "*   ```number of bedrooms``` - количество спален\n",
        "*   ```4digit postcode``` - 4-хзначный почтовый индекс"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "from pyspark.sql import Window\n",
        "from pyspark.sql import functions as F \n",
        "from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType\n",
        "from pyspark.sql.types import IntegerType\n",
        "\n",
        "import pandas as pd\n",
        "import requests"
      ],
      "metadata": {
        "id": "bhJ6KWN-Sh5q"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# скачаем файл\n",
        "\n",
        "train_response = requests.get('https://drive.google.com/uc?id=1uF1CStEeuFUMiXgQ2mhn42Cs9Ax8IUm3')\n",
        "with open('raw_sales.csv', 'wb') as file:\n",
        "    file.write(train_response.content)\n",
        "data = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)\n",
        "data.show(5)\n",
        "data.printSchema()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Jt3sCQTmSh76",
        "outputId": "0b380e6d-ba18-41db-b543-3462817680ac"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "+-------------------+--------+------+------------+--------+\n",
            "|           datesold|postcode| price|propertyType|bedrooms|\n",
            "+-------------------+--------+------+------------+--------+\n",
            "|2007-02-07 00:00:00|    2607|525000|       house|       4|\n",
            "|2007-02-27 00:00:00|    2906|290000|       house|       3|\n",
            "|2007-03-07 00:00:00|    2905|328000|       house|       3|\n",
            "|2007-03-09 00:00:00|    2905|380000|       house|       4|\n",
            "|2007-03-21 00:00:00|    2906|310000|       house|       3|\n",
            "+-------------------+--------+------+------------+--------+\n",
            "only showing top 5 rows\n",
            "\n",
            "root\n",
            " |-- datesold: timestamp (nullable = true)\n",
            " |-- postcode: integer (nullable = true)\n",
            " |-- price: integer (nullable = true)\n",
            " |-- propertyType: string (nullable = true)\n",
            " |-- bedrooms: integer (nullable = true)\n",
            "\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "xisyFowtQgx-"
      },
      "source": [
        "## Задание 1\n",
        "Добавьте к таблице следующие поля:\n",
        "*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)\n",
        "*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)\n",
        "*  Стоимость последнего проданного дома до текущего\n"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# группируем данные по postcode и сортируем в порядке возрастания даты продажи\n",
        "window = Window.partitionBy('postcode').orderBy('datesold')\n",
        "\n",
        "# avg_price_before_sale - получим среднюю стоимость домов до текущей продажи\n",
        "# диапазон задан с помощью window.rowsBetween(Window.currentRow-10, Window.currentRow-1)\n",
        "# среднее найдено с помощью avg()\n",
        "df = (\n",
        "    data.withColumn('avg_price_before_sale', F.avg(F.col('price'))\n",
        "    .over(window.rowsBetween(Window.currentRow-10, Window.currentRow-1)))\n",
        ")\n",
        "# avg_price_after_sale - аналогично - средняя стоимость домов после текущей продажи\n",
        "df = (\n",
        "    df.withColumn('avg_price_after_sale', F.avg(F.col('price'))\n",
        "    .over(window.rowsBetween(Window.currentRow+1, Window.currentRow+10)))\n",
        ")\n",
        "# last_price_before_sale - получим стоимость последнего проданного дома до текущего\n",
        "# предыдущее значение в рамках окна получено с помощью lag(колонка, на_сколько_назад)\n",
        "# рамки окна определены с помощью over(window)\n",
        "df = df.withColumn('last_price_before_sale', F.lag(F.col('price'), 1).over(window))\n",
        "\n",
        "# фильтруем данные, чтобы оставить только проданные дома\n",
        "df = df.filter(F.col('propertyType') == 'house')\n",
        "\n",
        "# сделаем новые столбцы типом int (это пригодится позже для поиска уникальных значений в строке)\n",
        "df = df.selectExpr('datesold', 'postcode', 'price', 'propertyType', 'bedrooms',\n",
        "                   'cast(avg_price_before_sale as int) as avg_price_before_sale', \n",
        "                   'cast(avg_price_after_sale as int) as avg_price_after_sale',\n",
        "                   'last_price_before_sale')\n",
        "\n",
        "# и вот он результат\n",
        "#df.select('*').orderBy('postcode', 'datesold').show(10)\n",
        "\n",
        "# в итоговом выводе  с помощью filter() уберём строки с null\n",
        "(\n",
        "    df.filter(F.col('avg_price_before_sale').isNotNull())\n",
        "    .select('*').orderBy('postcode', 'datesold').show(10)\n",
        ")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "y69PpxepIJDe",
        "outputId": "1adb7b5d-9b88-4d2d-d753-72226e267133"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+\n",
            "|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+\n",
            "|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|\n",
            "|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|\n",
            "|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|\n",
            "|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|\n",
            "|2008-07-29 00:00:00|    2600| 927000|       house|       4|               520500|              805750|                765000|\n",
            "|2008-09-02 00:00:00|    2600|1380000|       house|       5|               571312|              715250|                927000|\n",
            "|2008-09-08 00:00:00|    2600| 740000|       house|       3|               661166|              756250|               1380000|\n",
            "|2008-09-17 00:00:00|    2600| 720000|       house|       3|               669050|              741750|                740000|\n",
            "|2008-09-22 00:00:00|    2600| 690000|       house|       4|               708350|              730550|                720000|\n",
            "|2008-11-18 00:00:00|    2600| 635000|       house|       3|               698350|              755050|                690000|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+\n",
            "only showing top 10 rows\n",
            "\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Qvh2x6_8YU3F"
      },
      "source": [
        "## Задание 2\n",
        "В итоге у вас таблица с колонками (или нечто похожее):\n",
        "*   price\n",
        "*   Среднегодовая цена\n",
        "*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)\n",
        "*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)\n",
        "*  Стоимость последнего проданного дома до текущего ((1 балл)\n",
        "*  и др.\n",
        "\n",
        "Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf."
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# udf - в функцию с помощью array(*df) приходит список\n",
        "@F.udf(returnType=IntegerType())\n",
        "def count_unique_udf(row):\n",
        "    return len(set(row))\n",
        "(\n",
        "    df.filter(F.col('avg_price_before_sale').isNotNull())\n",
        "    .withColumn('unique_values', count_unique_udf(F.array(*df))).show(10)\n",
        ")"
      ],
      "metadata": {
        "id": "GzvPnuqaTtzI",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "4948cc1e-846c-4dbf-b03a-a3f0b893eeaa"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+\n",
            "|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|unique_values|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+\n",
            "|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|            7|\n",
            "|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|            8|\n",
            "|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|            8|\n",
            "|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|            8|\n",
            "|2008-07-29 00:00:00|    2600| 927000|       house|       4|               520500|              805750|                765000|            8|\n",
            "|2008-09-02 00:00:00|    2600|1380000|       house|       5|               571312|              715250|                927000|            8|\n",
            "|2008-09-08 00:00:00|    2600| 740000|       house|       3|               661166|              756250|               1380000|            8|\n",
            "|2008-09-17 00:00:00|    2600| 720000|       house|       3|               669050|              741750|                740000|            8|\n",
            "|2008-09-22 00:00:00|    2600| 690000|       house|       4|               708350|              730550|                720000|            8|\n",
            "|2008-11-18 00:00:00|    2600| 635000|       house|       3|               698350|              755050|                690000|            8|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+\n",
            "only showing top 10 rows\n",
            "\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# pandas_udf - тип SCALAR возвращает одно значение для каждой строки\n",
        "@pandas_udf(IntegerType(), PandasUDFType.SCALAR)\n",
        "def count_unique_pandas_udf(series) -> int:\n",
        "    return series.apply(lambda row: len(set(row)))\n",
        "(\n",
        "    df.filter(F.col('avg_price_before_sale').isNotNull())\n",
        "    .withColumn('unique_values', count_unique_pandas_udf(F.array(*df))).show(10)\n",
        ")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "P2sfqUzcZeJM",
        "outputId": "bf447a19-ea70-4d80-8c17-9de1ebc16e54"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+\n",
            "|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|unique_values|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+\n",
            "|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|            7|\n",
            "|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|            8|\n",
            "|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|            8|\n",
            "|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|            8|\n",
            "|2008-07-29 00:00:00|    2600| 927000|       house|       4|               520500|              805750|                765000|            8|\n",
            "|2008-09-02 00:00:00|    2600|1380000|       house|       5|               571312|              715250|                927000|            8|\n",
            "|2008-09-08 00:00:00|    2600| 740000|       house|       3|               661166|              756250|               1380000|            8|\n",
            "|2008-09-17 00:00:00|    2600| 720000|       house|       3|               669050|              741750|                740000|            8|\n",
            "|2008-09-22 00:00:00|    2600| 690000|       house|       4|               708350|              730550|                720000|            8|\n",
            "|2008-11-18 00:00:00|    2600| 635000|       house|       3|               698350|              755050|                690000|            8|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+-------------+\n",
            "only showing top 10 rows\n",
            "\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "pmSZTI9PAwQb"
      },
      "source": [
        "# Задание 3\n",
        "SQL like case when или if elif else\n",
        "\n",
        "Создайте колонку, в которой в которой будет отображаться \"+\", \"-\" или \"=\", если \"Средняя стомость 10 проданных домов до текущего в том же районе\" больше, меньше или равно \"Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)\", соотвественно.\n",
        "\n",
        "Если одно из полей Null, запишите в эту колонку \"Нет данных\""
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# сделаем задание через функцию when(если TRUE, тогда)\n",
        "df = df.withColumn(\n",
        "    'sign',\n",
        "    F.when(\n",
        "        F.col('avg_price_before_sale').isNull() | \n",
        "        F.col('avg_price_after_sale').isNull(),\n",
        "        'Нет данных'\n",
        "    ).when(\n",
        "        F.col('avg_price_before_sale') > F.col('avg_price_after_sale'),\n",
        "        '-'\n",
        "    ).when(\n",
        "        F.col('avg_price_before_sale') < F.col('avg_price_after_sale'),\n",
        "        '+'\n",
        "    ).otherwise(\n",
        "        '='\n",
        "    )\n",
        ")\n",
        "# здесь уже не фильтруем null, чтоб посмотреть на \"Нет данных\" :)\n",
        "df.show()"
      ],
      "metadata": {
        "id": "0psn9uqHTulg",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "c33e4d0b-60cf-49db-9224-4c1c2364063e"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+----------+\n",
            "|           datesold|postcode|  price|propertyType|bedrooms|avg_price_before_sale|avg_price_after_sale|last_price_before_sale|      sign|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+----------+\n",
            "|2007-07-08 00:00:00|    2600| 327000|       house|       1|                 null|              708350|                  null|Нет данных|\n",
            "|2007-08-16 00:00:00|    2600| 790000|       house|       4|               327000|              698350|                327000|         +|\n",
            "|2007-12-05 00:00:00|    2600| 825000|       house|       3|               558500|              679350|                790000|         +|\n",
            "|2008-04-24 00:00:00|    2600| 292500|       house|       1|               564250|              786600|                315000|         +|\n",
            "|2008-06-19 00:00:00|    2600| 765000|       house|       5|               479750|              868450|                329000|         +|\n",
            "|2008-07-29 00:00:00|    2600| 927000|       house|       4|               520500|              805750|                765000|         +|\n",
            "|2008-09-02 00:00:00|    2600|1380000|       house|       5|               571312|              715250|                927000|         +|\n",
            "|2008-09-08 00:00:00|    2600| 740000|       house|       3|               661166|              756250|               1380000|         +|\n",
            "|2008-09-17 00:00:00|    2600| 720000|       house|       3|               669050|              741750|                740000|         +|\n",
            "|2008-09-22 00:00:00|    2600| 690000|       house|       4|               708350|              730550|                720000|         +|\n",
            "|2008-11-18 00:00:00|    2600| 635000|       house|       3|               698350|              755050|                690000|         +|\n",
            "|2008-11-18 00:00:00|    2600| 950000|       house|       3|               679350|              701050|                635000|         +|\n",
            "|2008-11-21 00:00:00|    2600| 730000|       house|       3|               742850|              729550|                950000|         -|\n",
            "|2008-12-22 00:00:00|    2600| 855000|       house|       3|               786600|              716250|                730000|         -|\n",
            "|2008-12-24 00:00:00|    2600|1057500|       house|       4|               839200|              641500|                855000|         -|\n",
            "|2009-01-20 00:00:00|    2600|1150000|       house|       4|               715250|              641500|                475000|         -|\n",
            "|2009-01-22 00:00:00|    2600| 575000|       house|       3|               756250|              684000|               1150000|         -|\n",
            "|2009-02-13 00:00:00|    2600| 880000|       house|       4|               730550|              690700|                578000|         -|\n",
            "|2009-03-17 00:00:00|    2600|1015000|       house|       4|               701050|              735200|                410000|         +|\n",
            "|2009-03-28 00:00:00|    2600| 722000|       house|       4|               729550|              744000|               1015000|         +|\n",
            "+-------------------+--------+-------+------------+--------+---------------------+--------------------+----------------------+----------+\n",
            "only showing top 20 rows\n",
            "\n"
          ]
        }
      ]
    }
  ]
}